--packages databricks:mosaic:0.22.0

# ======================================================
# JOB: Carga de áreas queimadas (Arquivo TIFF)
# ORIGEM: /Volumes/datamasters/raw/raw_inpe
# DESTINO: Tabela Delta datamasters.b_inep.m_area_queim
======================================================

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from datetime import datetime
import os

spark = SparkSession.builder.getOrCreate()

In [0]:
# ======================================================
# 1️⃣ - PARÂMETRO DE ENTRADA (DATA DE REFERÊNCIA)
# ======================================================
dbutils.widgets.text("data_referencia", "")
referencia = dbutils.widgets.get("data_referencia").strip()

if not referencia or len(referencia) != 6:
    raise ValueError("⚠️ Parâmetro 'data_referencia' deve ser informado no formato AAAAMM (ex: 202509).")

print(f"📅 Data de referência recebida: {referencia}")

In [0]:
# ======================================================
# 2️⃣ - DEFINIÇÃO DE CAMINHOS E ARQUIVO
# ======================================================
origem_base = "/Volumes/datamasters/raw/raw_inpe"
nome_arquivo = f"aqm1km_{referencia}.tif"
caminho_arquivo = os.path.join(origem_base, nome_arquivo)

print(f"📂 Procurando arquivo: {caminho_arquivo}")

In [0]:
# ======================================================
# 3️⃣ - VALIDAÇÃO DE EXISTÊNCIA DO ARQUIVO
# ======================================================
arquivos = [f.path for f in dbutils.fs.ls(origem_base) if nome_arquivo in f.path]

if not arquivos:
    raise FileNotFoundError(f"🚫 Arquivo não encontrado: {nome_arquivo} em {origem_base}")

print(f"✅ Arquivo encontrado: {arquivos[0]}")

In [0]:
# ======================================================
# 4️⃣ - LEITURA DO ARQUIVO TIFF (RASTER)
# ======================================================
# Requer biblioteca RasterFrames:
# Adicione em Libraries (Maven): org.locationtech.rasterframes:rasterframes_2.12:0.9.0

df_raster = (
    spark.read.format("raster")
    .option("bandIndexes", "1")  # lê a primeira banda
    .load(arquivos[0])
)

print("🛰️ Leitura do arquivo TIFF concluída.")

In [0]:
# ======================================================
# 5️⃣ - ENRIQUECIMENTO COM METADADOS
# ======================================================
df_final = (
    df_raster
    .withColumn("data_referencia", F.lit(referencia))
    .withColumn("data_carga", F.current_timestamp())
)

display(df_final.limit(5))

In [0]:
# ==============================
# 6  Carga MENSAL
# ==============================
tabela_delta = "datamasters.b_inep.m_area_queim"

try:
    (
        df_final.write
        .format("delta")
        .mode("append")
        .partitionBy("data_referencia")
        .saveAsTable(tabela_delta)
    )

    print(f"✅ Dados gravados com sucesso na tabela {tabela_delta}")

except Exception as e:
    print(f"⚠️ Nenhum arquivo mensal encontrado para {data_ref_fmt}: {str(e)}")